# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

The only way to get started is to get started - so let's grab our dependencies for the day!

> NOTE: Using this notebook as presented will incur a charge of ~$3USD from OpenAI usage.

- 🤝 Breakout Room #1
  1. Task 1: Installing Required Libraries
  2. Task 2: Set Environment Variables
  3. Task 3: Creating a Simple RAG Pipeline with LangChain v.0.2.0
  4. Task 4: Synthetic Dataset Generation for Evaluation using Ragas (Optional)

- 🤝 Breakout Room #2
  1. Task 1: Evaluating our Pipeline with Ragas
  2. Task 2: Testing OpenAI's Claim
  3. Task 3: Selecting an Advanced Retriever and Evaluating

> NOTE: This Notebook *does* contain a bonus challenge, outlined at the bottom of the notebook, which you can complete instead of the notebook for full marks on the assignment.

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room Part #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://python.langchain.com/v0.2/docs/versions/v0_2/) of LangChain v0.2.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [1]:
!pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai langchain-qdrant

We'll also get the "star of the show" today, which is Ragas!

In [2]:
!pip install -qU ragas

We'll be leveraging [QDrant](https://qdrant.tech/) again as our LangChain `VectorStore`.

We'll also install `pymupdf` and its dependencies which will allow us to load PDFs using the `PyMuPDFLoader` in the `langchain-community` package!

In [3]:
!pip install -qU qdrant-client pymupdf pandas

## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [2]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

## Task 3: Creating a Simple RAG Pipeline with LangChain v0.2.0

Building on what we've been learning, we'll be leveraging LangChain v0.2.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

- [`PyMuPDFLoader`](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.PyMuPDFLoader.html)

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.2.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [3]:
from langchain_community.document_loaders import PyMuPDFLoader

PDF_LINK = "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf"

loader = PyMuPDFLoader(PDF_LINK)
documents = loader.load()

In [4]:
documents[0].metadata

{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'page': 0,
 'total_pages': 195,
 'format': 'PDF 1.3',
 'title': 'The Pmarca Blog Archives',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': 'Mac OS X 10.10 Quartz PDFContext',
 'creationDate': "D:20150110020418Z00'00'",
 'modDate': "D:20150110020418Z00'00'",
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

- [`RecursiveCharacterTextSplitter`](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html#langchain-text-splitters-character-recursivecharactertextsplitter)

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

CHUNK_SIZE = 200
CHUNK_OVERLAP = 50

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = CHUNK_SIZE,
    chunk_overlap = CHUNK_OVERLAP,
    length_function = len,
)

documents = text_splitter.split_documents(documents)

#split_chunks = text_splitter.split_documents(documents)

Let's confirm we've split our document.

In [6]:
len(documents)

1864

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

- [`OpenAIEmbeddings`](https://api.python.langchain.com/en/latest/embeddings/langchain_openai.embeddings.base.OpenAIEmbeddings.html#langchain-openai-embeddings-base-openaiembeddings)

> NOTE: We are purposefully using an older embedding model to try and answer the guiding question: Is TE3 better than Ada-002?

In [7]:
from langchain_openai import OpenAIEmbeddings

EMBEDDING_MODEL = "text-embedding-ada-002"

embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)

#### Creating a QDrant VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

- [`Qdrant`](https://api.python.langchain.com/en/latest/qdrant/langchain_qdrant.qdrant.QdrantVectorStore.html#langchain_qdrant.qdrant.QdrantVectorStore)

> NOTE: You'll need to provide the embedding dimension for Ada-002!

In [8]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

LOCATION = ":memory:"
COLLECTION_NAME = "PMarca Blogs"
VECTOR_SIZE = 1536

In [9]:
qdrant_client = QdrantClient(LOCATION)

qdrant_client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=VECTOR_SIZE, distance=Distance.COSINE),
)

qdrant_vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name=COLLECTION_NAME,
    embedding=embeddings,
)

qdrant_vector_store.add_documents(documents)

['2754ee33ac834b2a99e08fd83fe80356',
 'd7d25d3f1dd742e989b5211a0688f26a',
 '3db03d0a9def4451a8cd7150864c06f7',
 '398c6397cdac45c8b0fc7d6a0bf07077',
 '9918e80ad2ab45c4b027716f99b1c92d',
 '7a53962f9c7e47478aee9ffcf9106bee',
 '9b46fcc8c0404e63b32fba165d8a3872',
 '5638148c0d594749a89550663070eafb',
 '5c913d87843649f1ac7194afb2c39c08',
 '501a120298214b57a2ae9942f7593994',
 'b2bee315a07f47a8b3d241820d9aa46c',
 'da23c3fcfd374982aad413ebcbbfd75b',
 '3b19869dc30e4875b0c82c57b7d802c4',
 '7086425d24d44de49f0a4ec2953e6a93',
 '626c5d94e90542d8bf33ca9d070d86cf',
 '471529beed43436e9f6f50a3292980c3',
 '36624e51e33a4577a609f4b4801c6b30',
 'c6c2388678cc4d848c420cfc0e1efdd0',
 'ba0822da613047d7aa380a05b265fc39',
 '7e27082ee804456bbd0368e0e965d94a',
 '3e0f5ee09ac44961988d33037e94da0f',
 'c8b8c642ef154eb38eb30416f91388d3',
 '829cd0b587504919ab1d2590fb68bd81',
 '45bab4f335ae4bb4ad7bad565f17acbd',
 'd9d15e6a9ab844df9ca19539c45b7232',
 'ee0af2f907b84c228f5d871b1e14bc75',
 '53328d3fd51547e4aba797071b963fbf',
 

####❓ Question #1:

List out a few of the techniques that Qdrant uses that make it performant.

> NOTE: Check the [documentation](https://qdrant.tech/documentation/overview/) for more information about QDrant!

HNSW: Qdrant is a fully-fledged vector database that speeds up the search process by using a graph-like structure to find the closest objects in sublinear time. So we don’t calculate the distance to every object from the database, but some candidates only. Thanks to Hierarchical Navigable Small World (HNSW) algorithm we are able to compare the distance to some of the objects from the database, not to all of them.

Payload index: Qdrant is similar to the index in conventional document-oriented databases. This index is built for a specific field and type, and is used for quick point requests by the corresponding filtering condition. The index is also used to accurately estimate the filter cardinality, which helps the query planning choose a search strategy.

Quantization Techniques: it equipted with Scalar, Product, and unique Binary Quantization features significantly reduce memory usage and improve search performance (40x) for high-dimensional vectors.

Memory Maps and IO Uring: it has effective on-disk storage options and low level hardware optimization.

#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [10]:
retriever = qdrant_vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [11]:
retrieved_documents = retriever.invoke("What is a rule of thumb for selecting an industry to invest in?")

In [12]:
for doc in retrieved_documents:
  print(doc)

page_content='the existing order — and make sure that those forces of change
have a reasonable chance at succeeding.
Second rule of thumb:
Once you have picked an industry, get right to the center of it' metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 125, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': '2b5ab7e653f2401db295601d5bba559b', '_collection_name': 'PMarca Blogs'}
page_content='Third rule:
In a rapidly changing Held like technology, the best place to
get experience when you’re starting out is in younger, high-
growth companies.' metadata={'source': 'https://d1lamhf6l6yk6d.cloud

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [13]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

/opt/anaconda3/envs/llmops-course-aie4/lib/python3.11/site-packages/langsmith/client.py:311: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/opt/anaconda3/envs/llmops-course-aie4/lib/python3.11/site-packages/langsmith/client.py:5402: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


In [14]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [15]:
from langchain.prompts import ChatPromptTemplate

template = """
Use the provide context to answer the provided user query. Only use the provided context to answer the query. If you do not know the answer, response with "I don't know".

Question:
{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [16]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

Let's test it out!

In [17]:
question = "What is a rule of thumb for selecting an industry to invest in?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

A rule of thumb for selecting an industry to invest in is to ensure that the forces of change within that industry have a reasonable chance of succeeding. Additionally, once you have picked an industry, it is advised to get right to the center of it, where the great opportunities can be found.


In [18]:
question = "What did Pink Floyd have to say about how to proceed when investing in a new industry?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

I don't know.
[Document(metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 15, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': '8e3faf82177a4851a660bf2437d45373', '_collection_name': 'PMarca Blogs'}, page_content='ask if you can call them again if things change.\nTrust me — they’d much rather be saying “yes” than “no” —\nthey need all the good investments they can get.\nSecond, consider the environment.'), Document(metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

## Task 4: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evaluating on every core metric today, but in order to do that - we'll need to create a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

In [21]:
loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

eval_documents = loader.load()

text_splitter_eval = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap = 50
)

eval_documents = text_splitter_eval.split_documents(eval_documents)

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data?

To aviod overfitting. if we only use one type of split, the model may overfit to specific types of input lengths and fail when applied to real-world data with varied document structures. Different splitting strategies can capture various levels of context within the document so it provides a more comprehensive evaluation and can help the application be more robust and generalized. 


In [22]:
len(eval_documents)

624

> NOTE: 🛑 Running this cell as presented will incur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step. **YOU CAN SKIP THIS STEP BY LOADING THE `.csv` DIRECTLY FROM OUR REPOSITORY.** 🛑

#### Optional: SDG for Evaluation

In [ ]:
# from ragas.testset.generator import TestsetGenerator
# from ragas.testset.evolutions import simple, reasoning, multi_context
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
# critic_llm = ChatOpenAI(model="gpt-4o-mini")
# embeddings = OpenAIEmbeddings()

# generator = TestsetGenerator.from_langchain(
#     generator_llm,
#     critic_llm,
#     embeddings
# )

# distributions = {
#     simple: 0.5,
#     multi_context: 0.4,
#     reasoning: 0.1
# }

# num_qa_pairs = 20 # You can reduce the number of QA pairs to 5 if you're experiencing rate-limiting issues

# testset = generator.generate_with_langchain_docs(eval_documents, num_qa_pairs, distributions)
# testset.to_pandas()

Let's look at the output and see what we can learn about it!

In [ ]:
# testset.test_data[0]

DataRow(question='What are the potential risks and challenges faced by a company that does not raise enough money?', contexts=['Here’s why you shouldn’t do that:\nWhat are the consequences of not raising enough money?\nNot raising enough money risks the survival of your company,\nfor the following reasons:\nFirst, you may have — and probably will have — unanticipated\nsetbacks within your business.\nMaybe a new product release slips, or you have unexpected\nquality issues, or one of your major customers goes bankrupt, or\na challenging new competitor emerges, or you get sued by a big\ncompany for patent infringement, or you lose a key engineer.\nSecond, the funding window may not be open when you need\nmore money.'], ground_truth='Not raising enough money risks the survival of your company, for the following reasons: First, you may have — and probably will have — unanticipated setbacks within your business. Maybe a new product release slips, or you have unexpected quality issues, or on

In [ ]:
# testset_df = testset.to_pandas()
# testset_df.to_csv("testset.csv")

#### PREFERRED: Download `.csv` from DataRepository

In [23]:
!git clone https://github.com/AI-Maker-Space/DataRepository.git

Cloning into 'DataRepository'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 87 (delta 24), reused 28 (delta 8), pack-reused 8 (from 1)
Receiving objects: 100% (87/87), 70.09 MiB | 10.10 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [24]:
!mv DataRepository/testset.csv .

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [19]:
import pandas as pd

test_df = pd.read_csv("testset.csv")

In [20]:
test_df

,Unnamed: 0,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,0,How does the tendency to avoid inconsistency c...,['Five: Inconsistency-Avoidance Tendency\n[Peo...,The tendency to avoid inconsistency contribute...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
1,1,What are some of the challenges faced by start...,['structure that any established company has.\...,"In a startup, it is easy for the code not to g...",simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
2,2,What factors should be considered when decidin...,['Part 2: Skills and education\n[Please read m...,The answer to given question is not present in...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
3,3,What should be valued when evaluating candidat...,"[""How to hire the best people you've\never wor...",The answer to given question is not present in...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
4,4,What are the consequences of not raising enoug...,['Here’s why you shouldn’t do that:\nWhat are ...,Not raising enough money risks the survival of...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
5,5,How does Structured Procrastination suggest us...,['like?\nStructured procrastination\nThis is a...,Structured Procrastination suggests that inste...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
6,6,What analogy is used to describe the layers of...,['as if it’s an onion. Just like you peel an o...,The analogy used to describe the layers of ris...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
7,7,How can Structured Procrastination be used to ...,['like?\nStructured procrastination\nThis is a...,Structured Procrastination suggests that inste...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
8,8,How is the quality of a startup's product defi...,['Let’s start by deXning terms.\nThe caliber o...,The quality of a startup's product in the tech...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
9,9,What role can a campus computer lab play in he...,"['undergrads to do some of the work, and being...",A campus computer lab can play a role in helpi...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True


In [21]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [22]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [23]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [24]:
response_dataset[0]

{'question': 'How does the tendency to avoid inconsistency contribute to people being reluctant to change?',
 'answer': "The tendency to avoid inconsistency contributes to people's reluctance to change because change itself is perceived as a form of inconsistency. This avoidance is rooted in the brain's inclination to maintain previous conclusions, loyalties, reputational identity, and commitments. As a result, individuals may find it challenging to embrace new ideas or identities, leading to a resistance to change until they are ready or until a new generation emerges that is more open to change.",
 'contexts': ['Five: Inconsistency-Avoidance Tendency\n[People are] reluctant to change, which is a form of inconsistency\navoidance. We see this in all human habits, constructive and',
  'less brain-blocked by its previous conclusions…\nOne corollary of Inconsistency-Avoidance Tendency is that a per-\nson making big sacriXces in the course of assuming a new identity',
  '[T]ending to be ma

# 🤝 Breakout Room Part #2

## Task 1: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [25]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

In [26]:
results = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/95 [00:00<?, ?it/s]

In [27]:
results

{'faithfulness': 0.7118, 'answer_relevancy': 0.8747, 'context_recall': 0.6360, 'context_precision': 0.7032, 'answer_correctness': 0.5630}

In [30]:
results_df['faithfulness'].mean()

0.7118421052631579

In [29]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,How does the tendency to avoid inconsistency c...,The tendency to avoid inconsistency contribute...,[Five: Inconsistency-Avoidance Tendency\n[Peop...,The tendency to avoid inconsistency contribute...,1.000000,0.943772,0.500000,0.805556,0.320374
1,What are some of the challenges faced by start...,Some challenges faced by startups in establish...,[ied and determined. Sales calls get made. The...,"In a startup, it is easy for the code not to g...",1.000000,0.991209,1.000000,1.000000,0.600231
2,What factors should be considered when decidin...,I don't know.,[including your formal education. So I will st...,The answer to given question is not present in...,0.000000,0.000000,1.000000,0.000000,0.195205
3,What should be valued when evaluating candidat...,"When evaluating candidates for a startup, it i...",[priate for your particular startup.\nWith a w...,The answer to given question is not present in...,1.000000,0.974912,1.000000,0.000000,0.181087
4,What are the consequences of not raising enoug...,Not raising enough money risks the survival of...,[Here’s why you shouldn’t do that:\nWhat are t...,Not raising enough money risks the survival of...,1.000000,0.991354,0.333333,0.833333,0.503574
5,How does Structured Procrastination suggest us...,Structured Procrastination suggests that inste...,[standing.)\nThe gist of Structured Procrastin...,Structured Procrastination suggests that inste...,1.000000,0.957711,1.000000,0.916667,0.996443
6,What analogy is used to describe the layers of...,The analogy used to describe the layers of ris...,[as if it’s an onion. Just like you peel an on...,The analogy used to describe the layers of ris...,1.000000,1.000000,1.000000,0.750000,0.891681
7,How can Structured Procrastination be used to ...,Structured Procrastination can be used to one'...,[standing.)\nThe gist of Structured Procrastin...,Structured Procrastination suggests that inste...,1.000000,0.987979,0.500000,0.805556,0.583912
8,How is the quality of a startup's product defi...,The quality of a startup's product in the tech...,[The quality of a startup’s pr\nproduct\noduct...,The quality of a startup's product in the tech...,1.000000,0.993645,0.000000,1.000000,0.651020
9,What role can a campus computer lab play in he...,A campus computer lab can play a significant r...,[What should I do while I’m in school?\nI’m a ...,A campus computer lab can play a role in helpi...,0.750000,0.979579,0.333333,0.750000,0.877352


## Task : Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #1:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

#### apply new embedding using model "text-embedding-3-small"

In [31]:
te3_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

#### set up vectordb using new embedding

In [32]:
qdrant_client.create_collection(
    collection_name=COLLECTION_NAME+"TE3",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

qdrant_vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name=COLLECTION_NAME+"TE3",
    embedding=te3_embeddings,
)

qdrant_vector_store.add_documents(documents)

['99e63225c64c40faa4aa278d4e57bf1e',
 'a813b61eb9534e2fbf8a8885f86e6273',
 '4b89ea46dbb8445aa21c816723bb5b20',
 'fa5cebbc23554bf4a79a2a31e4973faa',
 'f4d7fce84de0401fac85961c34c46969',
 '4347ae0357244d4498d9f67257ef1c61',
 '8d3071d8a3724c97bdd2367318da45f6',
 '0468c6ae045b45c3b33a39b723c11383',
 '7882489e032c4dfab59ebc24bf45ee7d',
 'fbf52383b44d4b75a1145840bbd49df4',
 '29684c870c624e26b4661531350ac5b1',
 '45b17d68efc741ed9c2a7c657bb52130',
 '4421fab5cb1e4f81bdff46a6877cf58e',
 '0b29a81bf613467eb8d58512a52b6bb0',
 '2a6d3c1b7a8b4cbc8fe3fe78896be06f',
 '6c7d3888924847729c96780886db321e',
 '1183b0bd52074372b3e4790971f4f13f',
 '8487cf2f48a848d181b5bc73b19e5dca',
 'd33bfa12441d4793975e7a044df303f2',
 'f26f5afad04940f7be0e98da6d175851',
 '1a27d49c435540099895c2b02e1f3311',
 '37c6cde14b9b46598dea6d7bfa7c63e6',
 '74928a5ea14748bea73e28e844da3603',
 '895326ba0b354ccab3311cd1cd508eea',
 'b55ad28eb2a3456d96db2feef4de8700',
 '3eb1d7a976cc41578dad72b9be8a2583',
 'b9436a8ce698409bbcdfd6d8e4e8567a',
 

#### set up retriever

In [33]:
te3_retriever = qdrant_vector_store.as_retriever()

#### LCEL equivalent. It creates a document processing chain which is responsible for processing documents retrieved during the retrieval phase and combining them for further question-answering or summarization tasks

In [35]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

#### LCEL equivalent. it creates a retrieval chain which can retrieve relevant documents from a document store based on a query and pass those documents into the previously created document chain for further processing or answering

In [36]:
from langchain.chains import create_retrieval_chain

te3_retrieval_chain = create_retrieval_chain(te3_retriever, document_chain)

#### loop thourgh the questions and get the corresponded answer and context for later evaluation

In [37]:
answers = []
contexts = []

for question in test_questions:
  response = te3_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

#### wrap the information in a Hugging Face dataset for use in the Ragas library

In [38]:
te3_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

#### run evaluation

In [39]:
te3_advanced_retrieval_results = evaluate(te3_response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/95 [00:00<?, ?it/s]

#### get evaluation results

In [40]:
te3_advanced_retrieval_results

{'faithfulness': 0.7975, 'answer_relevancy': 0.9703, 'context_recall': 0.6228, 'context_precision': 0.6243, 'answer_correctness': 0.6278}

#### merge it with previous evaluation to see changes

In [41]:
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'ADA'])
df_comparison = pd.DataFrame(list(te3_advanced_retrieval_results.items()), columns=['Metric', 'TE3'])

df_merged = pd.merge(df_baseline, df_comparison, on='Metric')

df_merged['Baseline -> TE3'] = df_merged['TE3'] - df_merged['ADA']

df_merged

,Metric,ADA,TE3,Baseline -> TE3
0,faithfulness,0.711842,0.797475,0.085633
1,answer_relevancy,0.874734,0.970282,0.095548
2,context_recall,0.635965,0.622807,-0.013158
3,context_precision,0.703216,0.624269,-0.078947
4,answer_correctness,0.563029,0.627816,0.064787


####❓ Question #3:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?

yes I mean it increaed faithfulness, answer_relevancy, and answer_correctness. 
Faithfulness and answer_relevancy are tie to generation part, it means TE3 is better than ADA in terms of generating more relevant and correct answers.

Answer correctness measures whether the generated answer is factually/semantically accurate. TE3 shows a reasonable improvement too.

TE3 falls behind ADA in context recall and context precision, which might indicate that while TE3 generates better answers, it sacrifices some efficiency in retrieving the optimal context.

## Task 5: Selecting an Advanced Retriever and Evaluating

#### 🏗️ Activity #2

While the changes that occured due to modifying the embedding model were desirable - you're now tasked with improving `context_recall`, or `context_precision` (or both!).

You'll follow these steps:

1. Reason about this list of Advanced Retrieval methods:
  - [Contextual Compression (Reranker)](https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/contextual_compression/)
  - [MultiQueryRetriever](https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/MultiQueryRetriever/)
  - [Parent Document Retriever](https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/parent_document_retriever/)
2. Select the method you think will be the most performant.
3. Implement that method.
4. Create a LCEL chain that utlizes the new Retriever method.
5. Evaluate this LCEL and compare to the TE3 results.

> NOTE: We will spend more time in Session 14 diving into advanced retrieval methods, this activity is only to serve as a basic introduction to the idea of component-wise improvements and how they might impact metrics.

In [42]:
from IPython.display import display, Markdown

def pretty_print(message: str) -> None:
    display(Markdown(f"```markdown\n{message}\n```"))

In [43]:
question = test_questions[0]
print(question)
response = te3_retrieval_chain.invoke({"input" : question})
pretty_print([context.page_content for context in response["context"]])
pretty_print(response["answer"])

How does the tendency to avoid inconsistency contribute to people being reluctant to change?


```markdown
['Five: Inconsistency-Avoidance Tendency\n[People are] reluctant to change, which is a form of inconsistency\navoidance. We see this in all human habits, constructive and', 'less brain-blocked by its previous conclusions…\nOne corollary of Inconsistency-Avoidance Tendency is that a per-\nson making big sacriXces in the course of assuming a new identity', '[T]ending to be maintained in place by the anti-change tendency\nof the brain are one’s previous conclusions, human loyalties, repu-\ntational identity, commitments…', 'tational identity, commitments…\nIt is easy to see that a quickly reached conclusion, triggered by\nDoubt-Avoidance Tendency, when combined with a tendency to']
```

```markdown
The tendency to avoid inconsistency contributes to people's reluctance to change by creating a resistance to altering established habits, beliefs, and identities. This inconsistency-avoidance tendency leads individuals to cling to their previous conclusions, loyalties, and commitments, making it difficult for them to embrace new ideas or identities. As a result, even when faced with the need for change, people may find it challenging to let go of their past and adapt to new circumstances, as doing so would require them to confront and potentially disrupt their established sense of self and stability.
```

In [55]:
# from langchain.retrievers import ContextualCompressionRetriever
# from langchain.retrievers.document_compressors import LLMChainExtractor
# from langchain_openai import OpenAI

# primary_qa_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
# compressor = LLMChainExtractor.from_llm(primary_qa_llm )
# compression_retriever = ContextualCompressionRetriever(
#     base_compressor=compressor, base_retriever=te3_retriever
# )

# compressed_docs = compression_retriever.invoke(question)

In [51]:
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor


redundant_filter = EmbeddingsRedundantFilter(embeddings=te3_embeddings)
relevant_filter = EmbeddingsFilter(embeddings=te3_embeddings, similarity_threshold=0.5)
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[redundant_filter, relevant_filter]
)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor, base_retriever=te3_retriever
)

compressed_docs = compression_retriever.invoke(question)

In [52]:
pretty_print([context.page_content for context in compressed_docs])

```markdown
['Five: Inconsistency-Avoidance Tendency\n[People are] reluctant to change, which is a form of inconsistency\navoidance. We see this in all human habits, constructive and', 'less brain-blocked by its previous conclusions…\nOne corollary of Inconsistency-Avoidance Tendency is that a per-\nson making big sacriXces in the course of assuming a new identity', '[T]ending to be maintained in place by the anti-change tendency\nof the brain are one’s previous conclusions, human loyalties, repu-\ntational identity, commitments…']
```

In [55]:
re_ranker_retrieval_chain = create_retrieval_chain(compression_retriever, document_chain)

In [56]:
question = test_questions[0]
print(question)
response = re_ranker_retrieval_chain.invoke({"input" : question})
pretty_print([context.page_content for context in response["context"]])
pretty_print(response["answer"])

How does the tendency to avoid inconsistency contribute to people being reluctant to change?


```markdown
['Five: Inconsistency-Avoidance Tendency\n[People are] reluctant to change, which is a form of inconsistency\navoidance. We see this in all human habits, constructive and', 'less brain-blocked by its previous conclusions…\nOne corollary of Inconsistency-Avoidance Tendency is that a per-\nson making big sacriXces in the course of assuming a new identity', '[T]ending to be maintained in place by the anti-change tendency\nof the brain are one’s previous conclusions, human loyalties, repu-\ntational identity, commitments…']
```

```markdown
The tendency to avoid inconsistency contributes to people's reluctance to change by creating a resistance to altering established habits, beliefs, and identities. This avoidance is rooted in the brain's inclination to maintain previous conclusions and commitments, which can lead to a reluctance to embrace new ideas or identities. As individuals make significant sacrifices to adopt a new identity, they may find it challenging to let go of their past loyalties and reputational identities, further reinforcing their resistance to change.
```

In [57]:
answers = []
contexts = []

for question in test_questions:
  response = re_ranker_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

In [58]:
re_ranker_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [59]:
re_ranker_advanced_retrieval_results = evaluate(re_ranker_response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/95 [00:00<?, ?it/s]

Exception raised in Job[24]: TimeoutError()


In [60]:
re_ranker_advanced_retrieval_results

{'faithfulness': 0.7130, 'answer_relevancy': 0.9190, 'context_recall': 0.5263, 'context_precision': 0.6287, 'answer_correctness': 0.6071}

In [61]:
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'ADA'])
df_t3 = pd.DataFrame(list(te3_advanced_retrieval_results.items()), columns=['Metric', 'TE3'])
df_re_ranker = pd.DataFrame(list(re_ranker_advanced_retrieval_results.items()), columns=['Metric', 'RERANKER'])


df_merged1 = pd.merge(df_baseline, df_t3, on='Metric')
df_merged2 = pd.merge(df_merged1, df_re_ranker, on='Metric')

df_merged2['Baseline -> TE3'] = df_merged2['TE3'] - df_merged2['ADA']
df_merged2['TE3 -> RERANKER'] = df_merged2['RERANKER'] - df_merged2['TE3']

df_merged2

,Metric,ADA,TE3,RERANKER,Baseline -> TE3,TE3 -> RERANKER
0,faithfulness,0.711842,0.797475,0.712991,0.085633,-0.084484
1,answer_relevancy,0.874734,0.970282,0.918967,0.095548,-0.051315
2,context_recall,0.635965,0.622807,0.526316,-0.013158,-0.096491
3,context_precision,0.703216,0.624269,0.628655,-0.078947,0.004386
4,answer_correctness,0.563029,0.627816,0.607050,0.064787,-0.020766


#### 🚧 BONUS CHALLENGE 🚧

> NOTE: Completing this challenge will provide full marks on the assignment, regardless of the complete of the notebook. You do not need to complete this in the notebook for full marks.

##### **MINIMUM REQUIREMENTS**:

1. Baseline `LCEL RAG` Application using `NAIVE RETRIEVAL`
2. Baseline Evaluation using `RAGAS METRICS`
  - [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
  - [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
  - [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
  - [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
  - [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)
3. Implement a `SEMANTIC CHUNKING STRATEGY`.
4. Create an `LCEL RAG` Application using `SEMANTIC CHUNKING` with `NAIVE RETRIEVAL`.
5. Compare and contrast results.

##### **SEMANTIC CHUNKING REQUIREMENTS**:

Chunk semantically similar (based on designed threshold) sentences, and then paragraphs, greedily, up to a maximum chunk size. Minimum chunk size is a single sentence.

Have fun!